In [14]:
import pandas as pd
import requests
from collections import OrderedDict
import numpy as np
from bs4 import BeautifulSoup
import urllib2
from datetime import datetime as dt


In [217]:
update_df=pd.read_csv('./update_list.csv')
update_df.company_code[7]

6758

In [218]:
company_code=str(update_df.company_code[7])

url_timeline = 'https://kabutan.jp/stock/kabuka?code='+company_code
url_timeline2 = 'https://kabutan.jp/stock/kabuka?code='+company_code+'&ashi=day&page=3'
url_result='https://kabutan.jp/stock/finance?code='+company_code
up_url='https://kabutan.jp/warning/?mode=2_1&market=1'
down_url='https://kabutan.jp/warning/?mode=2_2&market=1'
sss='https://kabutan.jp/tansaku/?mode=1_f_T-y_Max-eki%26z-eki_Best100'

株価時系列と周辺情報について

In [219]:
html = urllib2.urlopen(url_result)
soup = BeautifulSoup(html)

In [220]:
#company name
company_name_list=soup.find('h1').string.split("(")
company_name=company_name_list[0]

In [221]:
#industry
try:
    company_industry=soup.find_all("a",href="/themes/")[0].string
except:
    company_industry=None

In [222]:
#warning share brands
datasets_up = pd.io.html.read_html(up_url)
warning_df=datasets_up[2]

In [223]:
datasets_n = pd.io.html.read_html(url_timeline)
df_time_line=datasets_n[5]

In [224]:
datasets_n2 = pd.io.html.read_html(url_timeline2)
df_today=datasets_n2[4]
price_df=datasets_n2[5]
df_today.columns=['date','start','highest','lowest','end','diff','diff_percentage','trade_volume']
price_df.columns=['date','start','highest','lowest','end','diff','diff_percentage','trade_volume']

In [225]:
i = 1
last_date = dt.now()
start_date=dt(2018,12,17)
j=0
while (last_date-start_date).days > 0:
    new_url='https://kabutan.jp/stock/kabuka?code='+company_code+'&ashi=day&page='+str(i)
    datasets_timeline=pd.io.html.read_html(new_url)
    dataset_timeline=datasets_timeline[5]
    date_length=len(dataset_timeline)
    dataset_timeline.columns=['date','start','highest','lowest','end','diff','diff_percentage','trade_volume']
    #print(date_length)
    last_date=dt.strptime(dataset_timeline.date[date_length-1],'%y/%m/%d')
    if i ==1:
        existing_timeline=dataset_timeline.copy()
    elif i > 1:
        existing_timeline=pd.concat([existing_timeline,dataset_timeline],axis=0)
    i = i+1

existing_timeline=existing_timeline.reset_index(drop=True)
    
for j in range(0,len(existing_timeline)):
    #print(j)
    if (dt.strptime(existing_timeline.date[len(existing_timeline)-j-1],'%y/%m/%d')-start_date).days > 0:
        stop_j=j
        break

existing_timeline=existing_timeline.loc[0:len(existing_timeline)-stop_j]

In [226]:
price_df['company_code']=company_code
price_df['company_name']=company_name
price_df['company_industry']=company_industry

df_today['company_code']=company_code
df_today['company_name']=company_name
df_today['company_industry']=company_industry

existing_timeline['company_code']=company_code
existing_timeline['company_name']=company_name
existing_timeline['company_industry']=company_industry

In [227]:
df_today

,date,start,highest,lowest,end,diff,diff_percentage,trade_volume,company_code,company_name,company_industry
0,19/05/10,5395,5505,5351,5408,78,1.5,9821200,6758,ソニー,None


In [228]:
collect_dfs=pd.io.html.read_html(up_url)
collect_df=collect_dfs[2]

In [246]:
column_name=list(collect_df.columns)

column_name[0]='company_code'
column_name[1]='company_name'
collect_df.columns=column_name
collect_df=collect_df.loc[:,['company_code','company_name']]


In [254]:
new_collect=pd.DataFrame(columns=['company_code','last_financial','last_share_price'])
new_collect["company_code"]=collect_df["company_code"].astype('object')
new_collect["last_financial"]=None
new_collect["last_share_price"]=None

update_df2=pd.concat([update_df,new_collect],axis=0)
update_df2=update_df2[~update_df2.duplicated(subset='company_code',keep='first')]
update_df2=update_df2.reset_index(drop=True)
update_df2

,company_code,last_financial,last_share_price
0,7203,2018,19/05/09
1,9984,NaN,NaN
2,9432,NaN,NaN
3,6861,NaN,NaN
4,9437,NaN,NaN
5,8306,2018,19/05/09
6,9983,NaN,NaN
7,6758,2018,19/05/09
8,9434,NaN,NaN
9,4502,NaN,NaN


# 決算情報について

In [230]:

basic_url='http://www.ullet.com/'+company_code+'.html'


In [231]:
#Basic info
datasets_basic = pd.io.html.read_html(basic_url)
df_asset=datasets_basic[4]
df_profit=datasets_basic[6]
df_cf=datasets_basic[8]
df_roa=datasets_basic[10]
df_employee=datasets_basic[17]

In [232]:
finance_columns_list=[]
for i in range(len(df_asset.columns)-1):
    finance_columns_list.append(df_asset.columns[i+1].encode('utf-8')[:4])


In [233]:
list_columns=['category']
#list_columns=list_columns.extend(finance_columns_list)
list_columns = list_columns + finance_columns_list
list_columns

['category', '2014', '2015', '2016', '2017', '2018']

In [234]:

'''
df_asset.columns=['category','end_mar_2014','end_mar_2015','end_mar_2016','end_mar_2017','end_mar_2018']
df_profit.columns=['category','end_mar_2014','end_mar_2015','end_mar_2016','end_mar_2017','end_mar_2018']
df_cf.columns=['category','end_mar_2014','end_mar_2015','end_mar_2016','end_mar_2017','end_mar_2018']
df_roa.columns=['category','end_mar_2014','end_mar_2015','end_mar_2016','end_mar_2017','end_mar_2018']
df_employee.columns=['category','end_mar_2014','end_mar_2015','end_mar_2016','end_mar_2017','end_mar_2018']
'''


df_asset.columns=list_columns
df_profit.columns=list_columns
df_cf.columns=list_columns
df_roa.columns=list_columns
df_employee.columns=list_columns

df_roa.category=['Cash','Revenue','Net_income','Operating_Cash_flow','ROA']
df_employee.category=['Employees_independent','Employees_group','Age','Working_length','Salary_independent']
#df_employee.category=['Employees_independent','Employees_group','Age','Working_length','Salary_independent']
df_cf.category=['Operating_Cash_flow','Investing_Cash_flow','Financial_Cash_flow','Modification','Change_cash']
df_profit.category=['Sales','Total_Sales','Cost_of_Sales','Other_Cost','Total_Cost','Gross_Profit','Income_before_taxes','NULL','Net_income']
df_asset.category=['Cash','Other_Current_Assets','Property_equipment','Intangible_Assets','Investments','Net_Assets','Current_Liability','NonCurrent_Liability','Net_equithy','Net_Assets']

#df_roa

In [235]:
nin = u"\u4eba"
nenrei=u"\u6b73"
nen=u"\u5e74"
manen=u"\u4e07\u5186"

df_employee.loc[0]=df_employee.loc[0].str.replace(nin,'')
df_employee.loc[0]=df_employee.loc[0].str.replace(',','')
df_employee.loc[1]=df_employee.loc[1].str.replace(nin,'')
df_employee.loc[1]=df_employee.loc[1].str.replace(',','')
df_employee.loc[2]=df_employee.loc[2].str.replace(nenrei,'')
df_employee.loc[3]=df_employee.loc[3].str.replace(nen,'')
df_employee.loc[4]=df_employee.loc[4].str.replace(manen,'')
df_roa.loc[4]=df_roa.loc[4].str.replace('%','')

In [236]:
df_all=pd.concat([df_asset,df_profit.loc[df_profit.category!='NULL']],axis=0)
df_all=pd.concat([df_all,df_cf],axis=0)
df_all=pd.concat([df_all,df_roa],axis=0)
df_all=pd.concat([df_all,df_employee],axis=0)
df_all2=df_all.reset_index(drop=True)
df_all2=df_all2[~df_all2.duplicated(subset='category',keep='last')]
df_all2

,category,2014,2015,2016,2017,2018
1,Other_Current_Assets,3158420,3248488,3213115,3395580,3589767
2,Property_equipment,750010,739285,820818,758199,739470
3,Intangible_Assets,1367466,1203616,1222044,1106723,1057660
4,Investments,9011358,9693529,10433801,11439912,12092312
6,Current_Liability,11629816,12188527,12984897,13831614,14785720
7,NonCurrent_Liability,916648,712087,556605,681462,623451
8,Net_equithy,2787256,2933717,3131888,3147480,3656367
9,Net_Assets,15333720,15834331,16673390,17660556,19065538
10,Sales,7767266,8215880,8105712,7603250,8543982
11,Total_Sales,7767266,8215880,8105712,7603250,8543982


In [237]:
df_all3=pd.melt(df_all2, 'category', var_name='term')
df_all3

,category,term,value
0,Other_Current_Assets,2014,3158420
1,Property_equipment,2014,750010
2,Intangible_Assets,2014,1367466
3,Investments,2014,9011358
4,Current_Liability,2014,11629816
5,NonCurrent_Liability,2014,916648
6,Net_equithy,2014,2787256
7,Net_Assets,2014,15333720
8,Sales,2014,7767266
9,Total_Sales,2014,7767266


In [238]:
df_all3["Company_code"]=company_code

In [239]:
df_all3

,category,term,value,Company_code
0,Other_Current_Assets,2014,3158420,6758
1,Property_equipment,2014,750010,6758
2,Intangible_Assets,2014,1367466,6758
3,Investments,2014,9011358,6758
4,Current_Liability,2014,11629816,6758
5,NonCurrent_Liability,2014,916648,6758
6,Net_equithy,2014,2787256,6758
7,Net_Assets,2014,15333720,6758
8,Sales,2014,7767266,6758
9,Total_Sales,2014,7767266,6758


In [240]:
#update_df.loc[update_df.company_code==int(company_code)]=[company_code,list_columns[len(list_columns)-1],existing_timeline.date[0]]
update_df2.loc[update_df.company_code==int(company_code),'last_financial']=list_columns[len(list_columns)-1]
update_df2.loc[update_df.company_code==int(company_code),'last_share_price']=existing_timeline.date[0]

In [241]:
update_df2

,company_code,last_financial,last_share_price
0,7203,2018,19/05/09
1,9984,NaN,NaN
2,9432,NaN,NaN
3,6861,NaN,NaN
4,9437,NaN,NaN
5,8306,2018,19/05/09
6,9983,NaN,NaN
7,6758,2018,19/05/09
8,9434,NaN,NaN
9,4502,NaN,NaN


In [242]:
update_df2.to_csv('./update_list.csv',index=False)